https://github.com/naity/image_tabular/blob/master/siim_isic_integrated_model.ipynb

# Imports

In [17]:
import pandas as pd
import numpy as np
import torch
import tensorflow
import fastai
from fastai.tabular.all import *  
# use gpu by default if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Load Data

In [21]:
# Tabular Data
df_tab = pd.read_csv("../Data/final_dataset.csv")
df_tab = df_tab[:100]
df_tab = df_tab.drop(['likes','dislikes','comment_count'], axis=1)
df_tab 

,category_id,views,title_1,title_2,title_3,title_4,title_5,title_6,title_7,title_8,...,thumbnail_link_7,country,comments_disabled,ratings_disabled,video_error_or_removed,description_915,thumbnail_link_8,thumbnail_link_9,thumbnail_link_10,thumbnail_link_11
0,10.0,17158579.0,764.0,661.0,18.0,437.0,124.0,55.0,1170.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,23.0,1014651.0,6061.0,225.0,718.0,647.0,2093.0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,23.0,3191434.0,743.0,1764.0,777.0,1402.0,465.0,3789.0,943.0,944.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,24.0,2095828.0,52.0,2159.0,45.0,609.0,5450.0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10.0,33523622.0,549.0,662.0,338.0,22.0,59.0,27.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,24.0,150974.0,755.0,590.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,24.0,70448.0,695.0,1174.0,71.0,3.0,1237.0,379.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,25.0,135620.0,11.0,11.0,135.0,24.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,22.0,88061.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


https://towardsdatascience.com/integrating-image-and-tabular-data-for-deep-learning-9281397c7318

## Train - Test - Split

In [22]:
from sklearn.model_selection import train_test_split

Train_data, Test_data = train_test_split( df_tab , test_size=0.33, random_state=42)
print(f"Length of df_tab: {len(df_tab)} \nLength of Train_data: {len(Train_data)} \nLength of Test_data: {len(Test_data)}" )

Length of df_tab: 100 
Length of Train_data: 67 
Length of Test_data: 33


# Prepare Data

## Tabular Data

In [26]:
# sort in categorical or continipus variables
target = 'views'
cat_columns = df_tab.select_dtypes(include=['object']).columns.tolist()
cont_columns = df_tab.select_dtypes(include=[np.number]).columns.tolist()
procs = []

In [ ]:
# create dataloader

In [43]:
# Train TabularDataLoaders
train = TabularDataLoaders.from_df(Train_data,
                                  path="",
                                  cat_names=cat_columns,
                                  cont_names=cont_columns,
                                  procs=procs,
                                  y_names=target)

test = TabularDataLoaders.from_df(Test_data,
                                  path="",
                                  cat_names=cat_columns,
                                  cont_names=cont_columns,
                                  procs=procs,
                                  y_names=target)

# Add Train Data to TabularDataLoaders

dls = DataLoaders(train, test)
#data = train.dls.train_ds.new(Test_data)

In [33]:
data.show_batch()

# Define Tabular Model

In [44]:
# Create an appropriate learner (neural network) for the tabular data
learn = tabular_learner(dls, layers=[300, 200, 100, 50], metrics=rmse)

In [46]:
# Find the suitable learning rate for training
learn.lr_find()

TypeError: 'TabDataLoader' object is not subscriptable

In [ ]:
# Fit the model 

In [45]:

learn.lr_find(start_lr = 1e-05,end_lr = 1e+05, num_it = 100)
learn.recorder.plot()

TypeError: 'TabDataLoader' object is not subscriptable

In [ ]:

#Fitting data and training the network
learn.fit_one_cycle(25)

In [ ]:
# get embedding sizes of categorical data
emb_szs = tab_data.train.get_emb_szs()

# output size of the tabular model that will be concatenated with cnn model output
tab_out_sz = 8

# use fastai functions to get a tabular model
tabular_model = TabularModel(emb_szs, len(cont_names), out_sz=tab_out_sz, layers=[8], ps=0.2)
tabular_model

## Training

In [ ]:
# find learning rate to train the last layer group first 
learn.freeze()
learn.lr_find()
learn.recorder.plot()

In [ ]:
# train
learn.fit_one_cycle(10, 1e-4)

In [ ]:
# unfreeze all layer groups to train the entire model using differential learning rates
learn.unfreeze()
learn.fit_one_cycle(5, slice(1e-6, 1e-4))

## Prediction

In [ ]:
# make predictions for the test set
preds, y = learn.get_preds(DatasetType.Test)